In [3]:
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.

True

# Ollama

Local LLM, download from [ollama.com](https://ollama.com/download/linux).

In [4]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama2")

In [12]:
response = llm.invoke("Tell me a joke")
print(response)


Why was the math book sad? Because it had too many problems! 😂


In [7]:
for chunk in llm.stream("Tell me a joke"):
    print(chunk, end="", flush=True)

Why did the scarecrow win an award? Because he was outstanding in his field! 😄

In [34]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "When asked to tell a joke, only tell the joke and nothing else (no introduction, etc.)."),
    ("user", "Tell me a joke about {input}")
])

In [9]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()  # The output of LLM will be a `message` -> parse it to a string

In [35]:
chain = prompt | llm | output_parser

In [39]:
response = chain.invoke({"input": "programmers"})
print(response)

Sure! Here's a joke for you:

Why do programmers prefer dark mode?

Because light attracts bugs.


In [38]:
for chunk in chain.stream({"input": "programmers"}):
    print(chunk, end="", flush=True)

Why did the programmer break up with his girlfriend?
She kept trying to fix her own problems, but he couldn't compile a solution.

# OpenAI

Create a key (<https://platform.openai.com/api-keys>) and set it in `.env`

In [42]:
from langchain_openai import OpenAI
from langchain.callbacks import get_openai_callback

llm = OpenAI(model_name="gpt-3.5-turbo")
chain = prompt | llm | output_parser

In [43]:
with get_openai_callback() as token_usage:
    
    for chunk in chain.stream({"input": "programmers"}):
        print(chunk, end="", flush=True)

    print()
    print(token_usage)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: .... You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}